# <font color=red>레이어 구성</font>

### 원본

In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
import dgl.function as fn
from dgl.nn.functional import edge_softmax


class MLP(nn.Module):
	def __init__(
		self, 
		input_dim, 
		hidden_dim, 
		output_dim,
		bias=True,
		act=F.relu,
	):
		super().__init__()

		self.input_dim = input_dim
		self.hidden_dim = hidden_dim
		self.output_dim = output_dim

		self.act = act

		self.linear1 = nn.Linear(input_dim, hidden_dim, bias=bias)
		self.linear2 = nn.Linear(hidden_dim, output_dim, bias=bias)
	
	def forward(self, h):
		h = self.linear1(h)
		h = self.act(h)
		h = self.linear2(h)
		return h


class GraphConvolution(nn.Module):
	def __init__(
			self,
			hidden_dim,
			act=F.relu,
			dropout_prob=0.2,
		):
		super().__init__()

		self.act = act
		self.norm = nn.LayerNorm(hidden_dim)
		self.prob = dropout_prob
		self.linear = nn.Linear(hidden_dim, hidden_dim, bias=False)
	
	def forward(
			self, 
			graph, 
			training=False
		):
		h0 = graph.ndata['h']

		graph.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'u_'))
		h = self.act(self.linear(graph.ndata['u_'])) + h0
		h = self.norm(h)
			
		# Apply dropout on node features
		h = F.dropout(h, p=self.prob, training=training)

		graph.ndata['h'] = h
		return graph


class GraphIsomorphism(nn.Module):
	def __init__(
			self,
			hidden_dim,
			act=F.relu,
			bias_mlp=True,
			dropout_prob=0.2,
		):
		super().__init__()

		self.mlp = MLP(
			input_dim=hidden_dim,
			hidden_dim=4*hidden_dim,
			output_dim=hidden_dim,
			bias=bias_mlp,
			act=act
		)
		self.norm = nn.LayerNorm(hidden_dim)
		self.prob = dropout_prob
	
	def forward(
			self, 
			graph, 
			training=False
		):
		h0 = graph.ndata['h']

		graph.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'u_'))
		h = self.mlp(graph.ndata['u_']) + h0
		h = self.norm(h)
			
		# Apply dropout on node features
		h = F.dropout(h, p=self.prob, training=training)

		graph.ndata['h'] = h
		return graph


class GraphIsomorphismEdge(nn.Module):
	def __init__(
			self,
			hidden_dim,
			act=F.relu,
			bias_mlp=True,
			dropout_prob=0.2,
		):
		super().__init__()

		self.norm = nn.LayerNorm(hidden_dim)
		self.prob = dropout_prob
		self.mlp = MLP(
			input_dim=hidden_dim,
			hidden_dim=4*hidden_dim,
			output_dim=hidden_dim,
			bias=bias_mlp,
			act=act,
		)
	
	def forward(
			self, 
			graph, 
			training=False
		):
		h0 = graph.ndata['h']

		graph.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'neigh'))
		graph.update_all(fn.copy_edge('e_ij', 'm_e'), fn.sum('m_e', 'u_'))
		u_ = graph.ndata['neigh'] + graph.ndata['u_']
		h = self.mlp(u_) + h0
		h = self.norm(h)
			
		# Apply dropout on node features
		h = F.dropout(h, p=self.prob, training=training)

		graph.ndata['h'] = h
		return graph


class GraphAttention(nn.Module):
	def __init__(
			self,
			hidden_dim,
			num_heads=4,
			bias_mlp=True,
			dropout_prob=0.2,
			act=F.relu,
		):
		super().__init__()

		self.mlp = MLP(
			input_dim=hidden_dim,
			hidden_dim=2*hidden_dim,
			output_dim=hidden_dim,
			bias=bias_mlp,
			act=act,
		)
		self.hidden_dim = hidden_dim
		self.num_heads = num_heads
		self.splitted_dim = hidden_dim // num_heads

		self.prob = dropout_prob

		self.w1 = nn.Linear(hidden_dim, hidden_dim, bias=False)
		self.w2 = nn.Linear(hidden_dim, hidden_dim, bias=False)
		self.w3 = nn.Linear(hidden_dim, hidden_dim, bias=False)
		self.w4 = nn.Linear(hidden_dim, hidden_dim, bias=False)
		self.w5 = nn.Linear(hidden_dim, hidden_dim, bias=False)
		self.w6 = nn.Linear(hidden_dim, hidden_dim, bias=False)

		self.act = F.elu
		self.norm = nn.LayerNorm(hidden_dim)
	
	def forward(
			self, 
			graph, 
			training=False
		):
		h0 = graph.ndata['h']
		e_ij = graph.edata['e_ij']

		graph.ndata['u'] = self.w1(h0).view(-1, self.num_heads, self.splitted_dim)
		graph.ndata['v'] = self.w2(h0).view(-1, self.num_heads, self.splitted_dim)
		graph.edata['x_ij'] = self.w3(e_ij).view(-1, self.num_heads, self.splitted_dim)

		graph.apply_edges(fn.v_add_e('v', 'x_ij', 'm'))
		graph.apply_edges(fn.u_mul_e('u', 'm', 'attn'))
		graph.edata['attn'] = edge_softmax(graph, graph.edata['attn'] / math.sqrt(self.splitted_dim))
	

		graph.ndata['k'] = self.w4(h0).view(-1, self.num_heads, self.splitted_dim)
		graph.edata['x_ij'] = self.w5(e_ij).view(-1, self.num_heads, self.splitted_dim)
		graph.apply_edges(fn.v_add_e('k', 'x_ij', 'm'))

		graph.edata['m'] = graph.edata['attn'] * graph.edata['m']
		graph.update_all(fn.copy_edge('m', 'm'), fn.sum('m', 'h'))
		
		h = self.w6(h0) + graph.ndata['h'].view(-1, self.hidden_dim)
		h = self.norm(h)

		# Add and Norm module
		h = h + self.mlp(h)
		h = self.norm(h)
			
		# Apply dropout on node features
		h = F.dropout(h, p=self.prob, training=training)

		graph.ndata['h'] = h 
		return graph


### 주석

In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
import dgl.function as fn
from dgl.nn.functional import edge_softmax


# Multi Layer Perceptron (GIN, GAT에서 사용할 수 있음)
class MLP(nn.Module):
    def __init__(
        self, 
        input_dim, 
        hidden_dim, 
        output_dim,
        bias=True,
        act=F.relu,
    ):
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.act = act

        # bias=bias: True
        self.linear1 = nn.Linear(input_dim, hidden_dim, bias=bias)
        self.linear2 = nn.Linear(hidden_dim, output_dim, bias=bias)
    
    def forward(self, h):
        h = self.linear1(h)
        h = self.act(h)
        h = self.linear2(h)
        return h

# Graph Convolutional Layer
class GraphConvolution(nn.Module):
    def __init__(
            self,
            hidden_dim,
            act=F.relu,
            dropout_prob=0.2,
        ):
        super().__init__()

        self.act = act
        self.norm = nn.LayerNorm(hidden_dim)
        self.prob = dropout_prob
		# linear transformation 하기위해 [Linear layer]선언 
        self.linear = nn.Linear(hidden_dim, hidden_dim, bias=False)
            
    def forward(
            self, 
            graph, 
            training=False
        ):
        # h0: 처음 graph의 각 node들의 data
        h0 = graph.ndata['h']

		# 이론 PPT 87p 식:  σ(ΣHW)
        # GCN은 주변에 있는 node feature들을 aggregation하는 것. 
        # ΣHW: node feature들(h)을 --copy--> m이라는 variable / m이라는 variable 들을 전부 --sum-->  u_라는 variable
        graph.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'u_'))
		# σ(ΣHW) + h0 (+ h0: residual connection)
        # u_라는 variable이 graph.ndata에 있을 것이고 이것을 [linear layer]에 넣은 후 activation 한게 Graph Convolution연산
        # 거기에다가 원래있던 node h0를 더하는 residual connection
        h = self.act(self.linear(graph.ndata['u_'])) + h0
        # layer normalization: optimization을 안정화하는데 도움이 됨
        h = self.norm(h)
        	
        ### Apply dropout on node features
        # 업데이트한 node feature에 dropout을 쓸지 말지 (training이여서 dropout을 틀지) 결정
        h = F.dropout(h, p=self.prob, training=training)

        # 업데이트 된 node feature들 다시 graph.ndata에 업데이트하여 graph의 node feature들이 업데이트
        graph.ndata['h'] = h
        # 업데이트 된 graph 리턴
        return graph

# (GCN)Linear layer를 쓰냐 (GIN)MLP를 쓰냐만 차이임.
class GraphIsomorphism(nn.Module):
    def __init__(
            self,
            hidden_dim,
            act=F.relu,
            bias_mlp=True,
            dropout_prob=0.2,
        ):
        super().__init__()
        
		# non-linear transformation 하기위해 [MLP] 선언
        self.mlp = MLP(
            input_dim=hidden_dim,
            hidden_dim=4*hidden_dim,
            output_dim=hidden_dim,
            bias=bias_mlp,
            act=act
        )
        self.norm = nn.LayerNorm(hidden_dim)
        self.prob = dropout_prob

    def forward(
            self, 
            graph, 
            training=False
        ):
        h0 = graph.ndata['h']

        graph.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'u_'))
		# aggregation된 message를 Linear layer가 아니라 [MLP]에 넣음
        h = self.mlp(graph.ndata['u_']) + h0
        h = self.norm(h)
        	
        # Apply dropout on node features
        h = F.dropout(h, p=self.prob, training=training)

        graph.ndata['h'] = h
        return graph
        
            
# GIN_E: message를 전달할 때 neighbor node feature들만 받을 수도 있지만, 어떤 edge feature를 통해서 aggregation 되었나도 같이 고려할 수 있음.
class GraphIsomorphismEdge(nn.Module):
    def __init__(
            self,
            hidden_dim,
            act=F.relu,
            bias_mlp=True,
            dropout_prob=0.2,
        ):
        super().__init__()

        self.norm = nn.LayerNorm(hidden_dim)
        self.prob = dropout_prob
		# non-linear transformation 하기위해 [MLP] 선언
        self.mlp = MLP(
            input_dim=hidden_dim,
            hidden_dim=4*hidden_dim,
            output_dim=hidden_dim,
            bias=bias_mlp,
            act=act,
        )
        
    def forward(
            self, 
            graph, 
            training=False
        ):
        h0 = graph.ndata['h']

		# initial node feature들(h0)을 m에 copy해서 neighbor node feature들을 sum aggregation한 것을 neigh라고 하자
        graph.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'neigh'))
		# 예를들어 center node에 달린 edge의 edge feature들을 전부 copy후 sum(:sum한게 message에 들어오는 것들임)
        graph.update_all(fn.copy_edge('e_ij', 'm_e'), fn.sum('m_e', 'u_'))
		# 주변의 aggregation node feature들 +  aggregation한 edge feature들 (concat한 것과 같은 효과)
        # 주변의 node feature들만 aggregation한 게 아니라 주변 neighbor들에 달려있는 edge, 그 edge가 가지고 있는 edge feature들 까지 같이 sum aggregaton한 것
        u_ = graph.ndata['neigh'] + graph.ndata['u_']
        # aggregation된 message를 non-linear transformation[MLP]하여 residual conncection(원래 node feature를 더함)
        h = self.mlp(u_) + h0
        h = self.norm(h)
        
        # Apply dropout on node features
        h = F.dropout(h, p=self.prob, training=training)

        graph.ndata['h'] = h
        return graph

# 이론 PPT 112p
class GraphAttention(nn.Module):
    def __init__(
            self,
            hidden_dim,
            num_heads=4,
            bias_mlp=True,
            dropout_prob=0.2,
            act=F.relu,
        ):
        super().__init__()

        self.mlp = MLP(
            input_dim=hidden_dim,
            hidden_dim=2*hidden_dim,
            output_dim=hidden_dim,
            bias=bias_mlp,
            act=act,
        )
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        # multi head dimension을 하되 맨처음의 hidden dimension과 맨 나중의 output dimension이 같도록
        # attention의 head는 num_heads니까 각 head에서 계산이 되는 dimension이 원래 hidden dimention 나누기 num_head(head의 개수)가 되어야함.
        self.splitted_dim = hidden_dim // num_heads

        self.prob = dropout_prob

        # linear transformation을 하되 bias는 쓰지 않음
        self.w1 = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.w3 = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.w4 = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.w5 = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.w6 = nn.Linear(hidden_dim, hidden_dim, bias=False)

        self.act = F.elu
        self.norm = nn.LayerNorm(hidden_dim)
            
    def forward(
            self, 
            graph, 
            # 밑에 dropout이 있어서 써놓음. training phase면 True. dropout을 킴.
            training=False
        ):
        # h0: 처음 graph의 각 node들의 data
        h0 = graph.ndata['h']
        e_ij = graph.edata['e_ij']
        
		### ai,j 구하는 과정
        # u: source node, v: neighbor node, x_ij: edge feature
        # <w1(h0)> source node에 linear transformation 
        # -1은 batch size 의미.
        graph.ndata['u'] = self.w1(h0).view(-1, self.num_heads, self.splitted_dim)
        # <w2(h0)> neighbor node에 linear transformation
        graph.ndata['v'] = self.w2(h0).view(-1, self.num_heads, self.splitted_dim)
        # edge feature의 dimension은 원래 6인데, 위의 graph.ndata['v']와 더하려면 dimension이 같아야 하므로 
        # <w3(e_ij)> edge feature에 linear tramsformation으로 dimension을 바꿔주고 
        graph.edata['x_ij'] = self.w3(e_ij).view(-1, self.num_heads, self.splitted_dim)

        # neighbor node 'v'에다가 edge feature 'x_ij'를 더해서 결과물을 'm'(message)이라고 하고 이 연산을 edge에다가 apply 하라.
        graph.apply_edges(fn.v_add_e('v', 'x_ij', 'm'))
        # source node 'u'와 'm'을 내적해서 결과물을 'attn'(attention)이라고 하고 이 연산을 edge에다가 apply 하라. 
        graph.apply_edges(fn.u_mul_e('u', 'm', 'attn'))
        # 이 'attn' edge data를 루트 d로 나눠 scaling하고 softmax 함
        graph.edata['attn'] = edge_softmax(graph, graph.edata['attn'] / math.sqrt(self.splitted_dim))
        

        ### xi' 구하는 과정
        # <w4(h0)> 원래의 neighbor node에 linear transformation 한 것
        graph.ndata['k'] = self.w4(h0).view(-1, self.num_heads, self.splitted_dim)
        # <w5(e_ij)> 원래의 edge feature에 linear tramsformation 한 것
        graph.edata['x_ij'] = self.w5(e_ij).view(-1, self.num_heads, self.splitted_dim)
        # neighbor node 'k'에다가 edge feature 'x_ij'를 더해서 결과물을 'm'(message)이라고 하고 이 연산을 edge에다가 apply 하라.
        graph.apply_edges(fn.v_add_e('k', 'x_ij', 'm'))

        # 'attn'과 'm'을 각각 곱해서 더하는 linear combination 과정
        # 'attn' 과 'm'을 각각 곱한 것을 다시 'm'이라고 하고
        graph.edata['m'] = graph.edata['attn'] * graph.edata['m']
        # 모든 edge 'm'을 copy하여 전부 더하여 'h'로 반환한 후 모두 업데이트하여라.
        graph.update_all(fn.copy_edge('m', 'm'), fn.sum('m', 'h'))
        
        # 모든 업데이트한 데이터들을 
        # multi-head attention으로 인해 self.splitted_dim으로 dimesion을 쪼갰던 것을 다시 self.hidden_dim으로 합쳐주고 
        # 원래 node featrue에다가 linear transformation 한 것에 더해줌.
        h = self.w6(h0) + graph.ndata['h'].view(-1, self.hidden_dim)
        h = self.norm(h)

		# Add and Norm module
		h = h + self.mlp(h)
		h = self.norm(h)
			
		# Apply dropout on node features
		h = F.dropout(h, p=self.prob, training=training)

		graph.ndata['h'] = h 
		return graph
